In [ ]:
# Sdandard AE

import torch
import numpy as np
import torch.nn as nn
import torch.utils.data as data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import tensorflow as tf
import pandas as pd

# Define the data structure
class MyDataset(Dataset):
    def __init__(self,filepath):
        xy = np.loadtxt(filepath,delimiter=',',dtype=np.float32)
        self.len = xy.shape[0]
        
        # load the data after the second column of the dataset (RSS data)
        self.x_data = torch.from_numpy(xy[:,2:])
        # load the first two columns of the dataset (Coordinate)
        self.y_data = torch.from_numpy(xy[:,0:2])
        
        #print("The data has prepared...")
        
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    
    def __len__(self):
        return self.len

# Initialize the positioning error
error_0 = [[]for i in range(15)]

realization = 500 # number of realizations
for z in range(realization):
    print(z)
    # Settings
    epochs = 1500
    batch_size = 14
    lr = 0.008 # learning rate
    k=1
    
    # load traning and testing dataset
    train_file = "./dataset/03_14_train.csv"
    test_file = "./dataset/03_10_test.csv"
    mydataset_train = MyDataset(train_file)
    mydataset_test = MyDataset(test_file)
    train_loader = DataLoader(dataset=mydataset_train,
                             batch_size=batch_size,
                             shuffle=True,
                             num_workers=2)
    test_loader = DataLoader(dataset=mydataset_test,
                             batch_size=10,
                             shuffle=True,
                             num_workers=2)

    # Model structure
    class AutoEncoder(nn.Module):
        def __init__(self):
            super(AutoEncoder, self).__init__()

            # Encoder
            self.encoder = nn.Sequential(
                nn.Linear(64, 48),
                nn.Tanh(),
                nn.Linear(48, 32),
                nn.Tanh(),
                nn.Linear(32, 16),
                nn.Tanh(),
                nn.Linear(16, 2),
            )

            # Decoder
            self.decoder = nn.Sequential(
                nn.Linear(2, 16),
                nn.Tanh(),
                nn.Linear(16, 32),
                nn.Tanh(),
                nn.Linear(32, 48),
                nn.Tanh(),
                nn.Linear(48, 64),
                nn.Tanh()
            )

        def forward(self, inputs):
            codes = self.encoder(inputs)
            decoded = self.decoder(codes)

            return codes, decoded

    #Loss Function
    class My_loss(nn.Module):
        def __init__(self):
            super().__init__()

        def forward(self, x, y):
            return torch.mean(torch.sqrt((torch.pow((30*x - 30*y), 2)).sum(axis=1)))

    # Optimizer and loss function
    model = AutoEncoder()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.MSELoss()
    loss_function_locate = My_loss()

    # Train
    for epoch in range(epochs):
        for data, labels in train_loader:
            inputs = data.view(-1, 64) 
            locate = labels.view(-1,2)
            # Forward
            codes, decoded = model(inputs)
            # Backward
            optimizer.zero_grad()
            # Calculate loss
            loss = loss_function(decoded, inputs)+loss_function_locate(codes, locate)
            loss.backward()
            optimizer.step()

        #Test
        if (epoch+1)%100 == 0 and epoch > 10:
            print(epoch)
            for data, labels in test_loader:
                inputs = data.view(-1, 64) 
                locate = labels.view(-1,2)
                # Forward
                codes, decoded = model(inputs)
                locate = 30*locate.detach().numpy()
                codes = 30*codes.detach().numpy()
                c=(locate-codes)*(locate-codes)
                d=np.sqrt(c.sum(axis=1))
                e=np.mean(d)
            # positioning error
            error_0[int((epoch+1)/100-1)].append(e)
            
# Save the result
for i in range(15):
    outcome_file = "./outcome/pure_AE_"+str(100+i*100)+".csv"  
    error = [error_0[i]]
    error = np.array(error)
    data1 = pd.DataFrame(error.T)
    data1.to_csv(outcome_file,header=None,index=None)